In [1]:
import pandas as pd
import pyarrow
import dask.dataframe as dd
from dask_ml.preprocessing import StandardScaler


In [2]:
columns_needed = ['date_id', 'time_id', 'symbol_id', 'weight', 'feature_00', 'feature_01',
       'feature_02', 'feature_03', 'feature_04', 'feature_05', 'feature_06',
       'feature_07', 'feature_08', 'feature_09', 'feature_10', 'feature_11',
       'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16',
       'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21',
       'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26',
       'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31',
       'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36',
       'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41',
       'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46',
       'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51',
       'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56',
       'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_61',
       'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66',
       'feature_67', 'feature_68', 'feature_69', 'feature_70', 'feature_71',
       'feature_72', 'feature_73', 'feature_74', 'feature_75', 'feature_76',
       'feature_77', 'feature_78','responder_6']


In [ ]:
'''TO-do
- Function to load datasets --> Use 
- Function to clean datasets
- Function to train XGB 
- Function to train SVM
    - Reduce data with PCA
- Display data


'''

In [10]:
# Dask Version 

# Load datasets to Dask dataframes
def load_datasets(file_number):
    file_path = f'/Users/elireadnour/Documents/Computer/Classes/machineLearning/MLFinal/jane-street-real-time-market-data-forecasting/train.parquet/partition_id={file_number}/part-0.parquet'
    new_dataset = dd.read_parquet(file_path, columns=columns_needed)
    return new_dataset

# Load data using Dask
data_0 = load_datasets(0)
data_1 = load_datasets(1)
data_2 = load_datasets(2)
data_3 = load_datasets(3)
data_4 = load_datasets(4)
data_5 = load_datasets(5)
data_6 = load_datasets(6)
data_7 = load_datasets(7)
data_8 = load_datasets(8)
data_9 = load_datasets(9)

In [11]:
# List of training datasets
training_datasets = [data_0,data_1,data_2,data_3, data_4, data_5, data_6, data_7, data_8, data_9]


In [12]:
# Dask Version 
import pyarrow.parquet as pq
import dask.dataframe as dd

# Load the Parquet file using PyArrow
table = pq.read_table(
    '/Users/elireadnour/Documents/Computer/Classes/machineLearning/MLFinal/jane-street-real-time-market-data-forecasting/lags.parquet/date_id=0/part-0.parquet'
)

# Convert PyArrow Table to a Pandas DataFrame and then to Dask
lag_0_pandas = table.to_pandas()
lag_0 = dd.from_pandas(lag_0_pandas, npartitions=4)


In [13]:
# Dask Version 
def optimize_dtypes_dask(df):
    # Convert float64 columns to float32
    float_cols = df.select_dtypes(include=['float64']).columns
    for col in float_cols:
        df[col] = df[col].astype('float32')
    
    # Convert int64 columns to int32
    int_cols = df.select_dtypes(include=['int64']).columns
    for col in int_cols:
        df[col] = df[col].astype('int32')
    
    return df

# Optimize Dask datasets
data_0 = optimize_dtypes_dask(data_0)
lag_0 = optimize_dtypes_dask(lag_0)


In [14]:
# DASK VERSION  

def get_lagged_data(previous_file, lag_column='responder_6', lag_name='responder_6_lag_1'):
    # Extract only the necessary columns and rename for clarity
    lagged_data = previous_file[['time_id', lag_column]].rename(columns={lag_column: lag_name})
    return lagged_data

def add_lags_to_next_file_dask(current_file, previous_file):
    # Get lagged data from the previous file
    lagged_data = get_lagged_data(previous_file)

    # Merge the lagged data into the current file using only time_id
    current_file = current_file.merge(lagged_data, on=['time_id'], how='left')

    return current_file

# Function to add lagged data to the current file
'''def add_lags_to_next_file_dask(current_file, previous_file):
    # Rename responder_6 in the previous file to avoid conflicts
    previous_file = previous_file.rename(columns={'responder_6': 'responder_6_lag_1'})

    # Drop existing lag column in the current file, if it exists
    if 'responder_6_lag_1' in current_file.columns:
        current_file = current_file.drop('responder_6_lag_1', axis=1)

    # Merge the previous day's lagged responder_6 into the current file
    current_file = current_file.merge(
        previous_file[['time_id', 'responder_6_lag_1']],
        on=['time_id'],
        how='left'
    )

    return current_file'''


# Assuming `data_0` and `lag_0` are Dask DataFrames
data_0 = data_0.merge(
    lag_0[['time_id', 'responder_6_lag_1']],
    on=['time_id'],
    how='left'
)
training_datasets[0] = data_0

# Iteratively add lag data to subsequent datasets
for x in range(1, len(training_datasets)):
    training_datasets[x] = add_lags_to_next_file_dask(training_datasets[x], training_datasets[x - 1])



The individual variables originally placed in the training datasets do not get changed. Once put in the list they are two different objects. Must reference the object via indexing the datasets list. 

In [16]:
for x in training_datasets:
    print(x.columns[-1])

responder_6_lag_1
responder_6_lag_1
responder_6_lag_1
responder_6_lag_1
responder_6_lag_1
responder_6_lag_1
responder_6_lag_1
responder_6_lag_1
responder_6_lag_1
responder_6_lag_1


PICK UP HERE TOMORROW 

In [17]:
data_1.head(1)

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_6
0,170,0,0,2.112212,NaN,NaN,NaN,NaN,NaN,1.06033,...,-1.249266,-0.353061,-0.944552,NaN,NaN,-0.218977,-0.304388,-0.272583,-0.421823,-0.044332


In [8]:
# Function to get and add lag files 
import pandas as pd
import numpy as np


values_for_lag = ['date_id', 'time_id', 'symbol_id','responder_6']

def add_lags_to_next_file(current_file, previous_file):
    # Rename responder_6 in the previous file to avoid conflicts
    previous_file = previous_file.rename(columns={'responder_6': 'responder_6_lag_1'})
    
    # Merge the previous day's lagged responder_6 into the current file
    current_file = current_file.merge(
        previous_file[['time_id', 'symbol_id', 'responder_6_lag_1']],
        on=['time_id', 'symbol_id'],
        how='left'
    )
    
    # Drop rows with NaN caused by lagging (optional)
    current_file = current_file.dropna()

    return current_file

data_0 = data_0.merge(lag_0[['time_id', 'symbol_id','responder_6_lag_1']], on=['time_id', 'symbol_id'], how='left')

for x in range(1,2):
    add_lags_to_next_file(training_datasets[x],training_datasets[x-1])


: 

In [ ]:
data_0.columns

Index(['date_id', 'time_id', 'symbol_id', 'weight', 'feature_00', 'feature_01',
       'feature_02', 'feature_03', 'feature_04', 'feature_05', 'feature_06',
       'feature_07', 'feature_08', 'feature_09', 'feature_10', 'feature_11',
       'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16',
       'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21',
       'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26',
       'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31',
       'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36',
       'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41',
       'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46',
       'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51',
       'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56',
       'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_6

In [8]:
# Anonymize
def anonymize(file): 
    pass
    return file

for x in training_datasets: 
    x = anonymize(x)

XGBoost

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np
from sklearn.preprocessing import StandardScaler


def scale_data(file):
    scaled_X = scale.fit_transform(file)
    return scaled_X


# Split the data into training and testing se
# Initialize the XGBoost Regressor
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
scale = StandardScaler()

def train_XGB(file,count):
# Train the model
    X = file.drop('responder_6', axis=1)
    y = file['responder_6']

    X_scaled = scale(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)

    '''mse = mean_squared_error(y_test, y_pred)
    print(f'data_{count} accuracy = {mse}')'''

    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f"MAPE: {mape}%")

    return xgb_model

counter = 0
for x in training_datasets: 
    train_XGB(x,counter)
    counter +=1


In [ ]:
# Predict on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model using Mean Squared Error (or other relevant metrics)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (XGBoost): {mse}")

In [137]:
from sklearn.model_selection import cross_val_score

# Using 5-fold cross-validation for model validation
kf_scores = cross_val_score(xgb_model, X, y, cv=5, scoring='neg_mean_squared_error')
print(f"Average Cross-Validation MSE: {-kf_scores.mean()}")


Average Cross-Validation MSE: 0.16485415697097777


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 3, 5, 7],
}

random_search = RandomizedSearchCV(XGBRegressor(objective='reg:squarederror'), param_distributions=param_dist, n_iter=30, cv=2, scoring='neg_mean_squared_error', verbose=2, random_state=42)
random_search.fit(X_train, y_train)

print(f"Best Parameters: {random_search.best_params_}")


Fitting 3 folds for each of 40 candidates, totalling 120 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   6.3s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   6.9s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   7.2s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, min_child_weight=3, n_estimators=300, subsample=0.8; total time=  11.7s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, min_child_weight=3, n_estimators=300, subsample=0.8; total time=  13.4s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=5, min_child_weight=3, n_estimators=300, subsample=0.8; total time=  10.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=5, min_child_weight=3, n_estimators=500, subsample=0.8; total

Tuned XGB Model using parameters from Randomized Search CV

In [ ]:
from xgboost import XGBRegressor

# Train the final model with the best parameters
final_xgb_model = XGBRegressor(
    objective='reg:squarederror',
    subsample=1.0,
    n_estimators=500,
    min_child_weight=3,
    max_depth=10,
    learning_rate=0.1,
    colsample_bytree=0.6,
    random_state=42
)

# Fit on the training data
final_xgb_model.fit(X_train, y_train)


In [139]:
from xgboost import XGBRegressor

# Train the final model with the best parameters
final_xgb_model = XGBRegressor(
    objective='reg:squarederror',
    subsample=1.0,
    n_estimators=500,
    min_child_weight=3,
    max_depth=10,
    learning_rate=0.1,
    colsample_bytree=0.6,
    random_state=42
)

# Fit on the training data
final_xgb_model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [140]:
from sklearn.metrics import mean_squared_error

# Make predictions on the test set
y_pred = final_xgb_model.predict(X_test)

# Calculate Mean Squared Error on the test set
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (Final Model): {mse}")


Mean Squared Error (Final Model): 0.10424154996871948


In [142]:
from sklearn.model_selection import cross_val_score

# Using 5-fold cross-validation for model validation
kf_scores = cross_val_score(final_xgb_model, X, y, cv=5, scoring='neg_mean_squared_error')
print(f"Average Cross-Validation MSE: {-kf_scores.mean()}")


Average Cross-Validation MSE: 0.1558243989944458


# Data Preprocessing

Data Preprocessing: Standard Scaller 

In [ ]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()

X = scale.fit_transform(X)



PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)  # Automaticall collect enough features to caputre 95 % of variance
X_pca = pca.fit_transform(X)

cumulative_variance = pca.explained_variance_ratio_.cumsum()
print(cumulative_variance)


[0.10730664 0.20812827 0.27430323 0.3348844  0.38985676 0.44306043
 0.4930155  0.5316344  0.5646317  0.5944145  0.62315035 0.6516101
 0.67846054 0.70308244 0.7267795  0.7499029  0.7708257  0.7903909
 0.8078895  0.8237381  0.8393095  0.8532956  0.86696416 0.88004553
 0.8927529  0.9019354  0.9109082  0.9195607  0.92790073 0.9355657
 0.9425222  0.9482943  0.9533449 ]


XGB w/PCA

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Initialize the XGBoost Regressor
xgb_model1 = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

# Train the model
xgb_model1.fit(X_train1, y_train1)

# Predict on the test set
y_pred1 = xgb_model1.predict(X_test1)

# Evaluate the model using Mean Squared Error (or other relevant metrics)
mse1 = mean_squared_error(y_test1, y_pred1)
print(f"Mean Squared Error (XGBoost): {mse1}")


Mean Squared Error (XGBoost): 0.32325002551078796


In [ ]:
from sklearn.model_selection import cross_val_score

# Using 5-fold cross-validation for model validation
kf_scores = cross_val_score(xgb_model, X_scaled, y, cv=5, scoring='neg_mean_squared_error')
print(f"Average Cross-Validation MSE: {-kf_scores.mean()}")


In [ ]:
# git add .
# git commit -m "commit message"
# git push origin main

Data Preprocessing
- Heat maps Variance
- XGBoost --> Finds most highly correlated data

Random Forest

Data Preprocessing
- Heat maps Variance
- XGBoost --> Finds most highly correlated data

Random Forest